# Manual básico para usar un bot de Telegram mediante la API de Python

#### Autor: Alejandro Martínez Valencia

### Instalando librerías necesarias

Antes que nada debemos instalar las siguientes librerías dos librerias. La primera es la encargada de manejar la API de Telegram para comunicarse con el bot y la segunda es una libreria auxiliar para manejar los tokens.

In [1]:
!pip3 install python-telegram-bot
!pip3 install python-dotenv


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


#### Importamos la libreria Telegram y auxiliares

In [2]:
import os
import json

import telegram
# from dotenv import load_dotenv

#### Utilizaremos la clase Bot de telegram
Al instanciar Bot, nos pedirá como parámetro el token de nuestro bot de Telegram. Para esto, previamente debimos crear un bot dentro Telegram con ayuda de [BotFather](https://core.telegram.org/bots), y obtener el token.

Para evitar exponer el token dentro del código de nuestra aplicación, o en producción o compartido en un repositorio público, es recomendable guardar este dato dentro de una variable de entorno e impórtala.

Para esta tarea, utilizaremos `load_dotenv`. Para más info, puedes checarlo [aquí](https://pypi.org/project/python-dotenv/)

In [3]:
# load_dotenv()
# TOKEN_TELEGRAM_BOT = os.getenv("TOKEN_TELEGRAM_BOT")
# TELEGRAM_USER = os.getenv("TELEGRAM_USER")
TOKEN_TELEGRAM_BOT = '7593613541:AAFPiHs10odKrgJ0pXcz7G-KJ7ectRLDoEk'
TELEGRAM_USER = '-4747972537'

In [4]:
bot = telegram.Bot(token=TOKEN_TELEGRAM_BOT)

#### Verificamos que se ha conectado a nuestro bot imprimiendo sus datos.

In [5]:
print(await bot.get_me())

User(can_connect_to_business=False, can_join_groups=True, can_read_all_group_messages=False, first_name='realadvisor', has_main_web_app=False, id=7593613541, is_bot=True, supports_inline_queries=False, username='realadvisor_bot')


### Para enviar un mensaje a un usuario, necesitamos conocer su chat_id. 
Para ello, desde Telegram, le enviamos un mensaje a nuestro bot y después ejecutamos el siguiente bloque:

In [11]:
updates = await bot.get_updates()
updates

(Update(message=Message(channel_chat_created=False, chat=Chat(first_name='German', id=2820988, last_name='Z', type=<ChatType.PRIVATE>, username='germanztz'), date=datetime.datetime(2025, 2, 12, 14, 6, 6, tzinfo=datetime.timezone.utc), delete_chat_photo=False, entities=(MessageEntity(length=6, offset=0, type=<MessageEntityType.BOT_COMMAND>),), from_user=User(first_name='German', id=2820988, is_bot=False, language_code='es', last_name='Z', username='germanztz'), group_chat_created=False, message_id=6, supergroup_chat_created=False, text='/start'), update_id=441774489),
 Update(message=Message(channel_chat_created=False, chat=Chat(api_kwargs={'all_members_are_administrators': True}, id=-4730516620, title='Real Advisor', type=<ChatType.GROUP>), date=datetime.datetime(2025, 2, 12, 14, 12, 32, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='German', id=2820988, is_bot=False, language_code='es', last_name='Z', username='germanztz'), group_chat_created=True, 

In [16]:
# Update(message=Message(channel_chat_created=False, chat=Chat(first_name='German', id=2820988, last_name='Z', type=<ChatType.PRIVATE>, username='germanztz'), date=datetime.datetime(2025, 2, 12, 18, 53, 30, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='German', id=2820988, is_bot=False, language_code='es', last_name='Z', username='germanztz'), group_chat_created=False, message_id=15, supergroup_chat_created=False, text='Test'), update_id=441774496))

print(updates[-1].message.chat.id)
print(updates[-1].message.date)
print(updates[-1].message.text)
# eval(str((await bot.get_updates())[-1].message.chat))

2820988
2025-02-12 18:53:30+00:00
Test


Para nuestra comodidad, podemos definir una función para enviar el mensaje. Sustituya la variable `TELEGRAM_USER` por su `id`.

In [8]:
async def enviar_mensaje(mensaje):
    """Enviamos un mensaje a un usuario con determinado chat_id"""
    await bot.sendMessage(
        text=mensaje, 
        chat_id=TELEGRAM_USER)

Comprobamos que se puede enviar el mensaje de texto

In [9]:
result = (await enviar_mensaje('Buenas noches, ya me iré a dormir'))
result

#### Mensajes con formato
Sí queremos enviar mensajes con algún tipo de formato, debemos enviarle modo de parseo que acepte Telegram. En este caso `Markdown` o `HTML`.

In [10]:
bot.send_message(
    text="*Buenas Noches*, ya me iré a dormir",
    chat_id=TELEGRAM_USER,
    parse_mode=telegram.ParseMode.MARKDOWN_V2
)

AttributeError: module 'telegram' has no attribute 'ParseMode'

In [ ]:
bot.send_message(
    text="*Texto en negritas*, _Texto en italica_, `Texto en monospace`, Un link: [Ábrelo](www.google.com.mx)",
    chat_id=TELEGRAM_USER,
    parse_mode=telegram.ParseMode.MARKDOWN_V2
)

In [ ]:
bot.send_message(
    text="<b>Buenas Noches</b>, ya me iré a dormir",
    chat_id=TELEGRAM_USER,
    parse_mode=telegram.ParseMode.HTML
)

#### Enviar una imagen
Para enviar una imagen que se encuentra localmente, nos auxiliamos de la función `open`.

In [ ]:
bot.send_photo(
    photo=open('meme_01.jpg', 'rb'),
    chat_id=TELEGRAM_USER
)

También se puede enviar una imagen desde una dirección url

In [ ]:
bot.send_photo(
    photo='https://static4.abc.es/media/bienestar/2020/03/13/gato-dormir-bien-nordico-kwtB--620x349@abc.jpg',
    chat_id=TELEGRAM_USER
)

#### Enviar un archivo de voz desde local
Aquí agregamos el parámetro `timeout` para esperar a que se cargue por completo el archivo que se enviará.

In [ ]:
bot.send_voice(
    voice=open('voice.ogg', 'rb'),
    chat_id=TELEGRAM_USER, 
    timeout=1000
)

#### Recuperar un audio, una foto o un archivo del último mensaje
Sí el usuario le envia un archivo, podemos recuperarlo verificando si nos envio un objeto multimedia, para posteriormente guardarlos de la siguente forma.

In [ ]:
# Verificamos el id del objeto que nos envió el usuario
file_id = bot.get_updates()[-1].message.voice.file_id
image = bot.get_updates()[-1].message.photo[-1]

# Obtenemos el archivo y lo descargamos
newFile = bot.get_file(file_id)
newFile.download('voice2.ogg')

#### Enviar un GIF

In [ ]:
bot.send_animation(
    chat_id=TELEGRAM_USER, 
    animation=open('amor.gif', 'rb'), 
    timeout=1000
);

#### Enviar un archivo de música

In [ ]:
bot.send_audio(
    audio=open('tests/test.mp3', 'rb'),
    chat_id=TELEGRAM_USER,
);

#### Enviar un archivo cualquiera

In [ ]:
bot.send_document(
    document=open('hola.txt', 'rb'),
    chat_id=TELEGRAM_USER
);

#### Enviar una imagen guardada en un Buffer de memoria
Muchas veces queremos enviar una imagen o gráfica producto de alguna ejecución al vuelo, pero no queremos guardarla en disco. Podemos enviar esa imagen almacenandola temporalmente en un buffer y enviarla a Telegram.

In [ ]:
# Importamos la libreria para guardar el buffer
from io import BytesIO

# Utilizaremos una gráfica de matplotlib como ejemplo
import numpy as np
from matplotlib.figure import Figure

Generaremos un gráfico de matplotlib

In [ ]:
x = np.arange(0, 10, 0.1)
y = np.cos(x)

fig = Figure()
ax = fig.subplots()
ax.plot(x, y)

Instanciamos el buffer. Le agregamos un nombre al objeto

In [ ]:
buffer = BytesIO()
buffer.name = 'image.jpg'

Con el método `savefig` de `Figure`, guardamos el gráfico dentro del objeto Buffer

In [ ]:
fig.savefig(buffer, format="jpg")

Posiciona el cursor  al inicio del buffer

In [ ]:
buffer.seek(0)

Envia la imagen contenida en el buffer

In [ ]:
bot.send_photo(
    chat_id=TELEGRAM_USER, 
    photo=buffer
);

#### Crear una lista de botones
Podemos crear y enviar una lista con botones para sugerir al user que contestarle al bot.

Personalizamos los botones en una cuadrícula

In [ ]:
botones = [
    ['/iniciar', '/ayuda'],
    ["Saludos"]
]

Formateamos los botones y se los enviamos al usuario

In [ ]:
reply_markup = telegram.ReplyKeyboardMarkup(botones)

bot.send_message(
    chat_id=TELEGRAM_USER,
    text="Te envio unos botones:",
    reply_markup=reply_markup
);

Para borrar los botones mostrados al usuario, lo enviamos:

In [ ]:
reply_markup = telegram.ReplyKeyboardRemove()

bot.send_message(
    chat_id=TELEGRAM_USER, 
    text="Ya borré los botones 💃",
    reply_markup=reply_markup
)

#### Configurar mensajes de espera del bot ("... Escribiendo", "... Grabando audio...", etc)
Se puede enviar una señal para que en la pantalla de Telegram pinté que el Bot está escribiendo, enviando una imagen, grabando un audio, etc.
La acción que se va a realizar se llama con un string. Existen estas posibilidades : 
- record_audio 
- record_video
- record_video_note 
- typing
- upload_audio
- upload_document
- upload_photo 
- upload_video 
- upload_video_note

Se envia ejecutando el siguiente método. 

In [ ]:
bot.send_chat_action(
    chat_id=TELEGRAM_USER,
    action="record_audio",
);

La acción se muestra al usuario durante 5 segundos, pero es útil para avisarle al usuario que el bot está contestando, pero estamos cargando una imagen o enviando un audio. 

Este método se puede agregar dentro de una función ya sea agregandola directamente o con un decorador, como sigue:

In [ ]:
def enviar_mensaje(mensaje, chat_id=TELEGRAM_USER):
    """Enviamos un mensaje a un usuario con determinado chat_id"""
    bot.send_chat_action(chat_id=chat_id, action='typing')
    bot.sendMessage(chat_id=chat_id, text=mensaje)

In [ ]:
enviar_mensaje("Buenas noches!!!")

### Comentarios Finales

Existen más funcionalidades avanzadas para utilizar un bot dentro de Telegram desde python, tal como la automatización de respuestas, o configurar juegos, etc. Puedes revisar la documentación completa en el repositorio de [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Introduction-to-the-API). En un futuro espero hacer una revisión de estás funcionalidades avanzadas en otro notebook. 

Ojalá este documento te ayude a implementar bots dentro de tus proyectos. 😉